In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.dummy import DummyClassifier
from sklearn.model_selection import cross_val_score, GridSearchCV, ParameterGrid, train_test_split
from sklearn import metrics

import pandas as pd
import numpy as np

from sklearn.neighbors import KNeighborsClassifier

# import warnings filter
from warnings import simplefilter
# ignore all future warnings
simplefilter(action='ignore', category=FutureWarning)

In [ ]:
dataset_path = 'allUsers.lcl.csv'

df = pd.read_csv(dataset_path)
#knn Cannot work with mining data, so replace them with a 0
df = df.replace('?', 0)
#df = df.dropna(thresh=len(df) * 0.10, axis=1)



# KNN

# Class recognition with User label

In [4]:
df2 = df.drop(['Class'], axis=1)
labels_Class = df['Class']


X_train, X_test, y_train, y_test,  = train_test_split(df2, labels_Class, test_size=0.20, random_state=42)


In [5]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=1)
knn.fit(X_train, y_train)


KNeighborsClassifier(n_neighbors=1)

In [6]:
print(knn.score(X_test, y_test))


0.9602432778489116


# Hand owner recognition with Class label

In [ ]:
df2 = df.drop(['User'], axis=1)
labels_Users = df['User']
X_train, X_test, y_train, y_test,  = train_test_split(df2, labels_Users, test_size=0.20, random_state=42)

In [ ]:
knn = KNeighborsClassifier(n_neighbors=1)
knn.fit(X_train, y_train)


KNeighborsClassifier(n_neighbors=1)

In [ ]:
print(knn.score(X_test, y_test))

0.8646606914212548


# Class recognition without User label

In [ ]:
df2 = df.drop(['User'], axis=1)
df2 = df2.drop(['Class'], axis=1)
labels_Class = df['Class']
X_train, X_test, y_train, y_test,  = train_test_split(df2, labels_Class, test_size=0.20, random_state=42)

In [ ]:
knn = KNeighborsClassifier(n_neighbors=1)
knn.fit(X_train, y_train)
print(knn.score(X_test, y_test))

0.9596670934699104


# Class recognition without Class label

In [ ]:
df2 = df.drop(['User'], axis=1)
df2 = df2.drop(['Class'], axis=1)

labels_Users = df['User']
X_train, X_test, y_train, y_test,  = train_test_split(df2, labels_Users, test_size=0.20, random_state=42)

In [ ]:
knn = KNeighborsClassifier(n_neighbors=1)
knn.fit(X_train, y_train)
print(knn.score(X_test, y_test))

0.8646606914212548


# SVM

In [23]:
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler

import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
!pip insall sklearn.grid_search
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix


dataset_path = 'allUsers.lcl.csv'

df = pd.read_csv(dataset_path)

df = df.replace('?', np.nan)
df = df.dropna(thresh=len(df) * 0.8 , axis=1)
df = df.replace(np.nan, 0)

drop_indices = np.random.choice(df.index, 75000, replace=False)
df = df.drop(drop_indices)
df

ERROR: unknown command "insall" - maybe you meant "install"


,Class,User,X0,Y0,Z0,X1,Y1,Z1,X2,Y2,Z2,X3,Y3,Z3,X4,Y4,Z4,X5,Y5,Z5
60,1,0,55.845720,66.522803,-68.410839,77.106544,36.734284,-75.072331,85.470310,62.473837,-77.605075,34.2457979030067,76.2252873173598,-61.5592919042357,62.4769143002536,5.03622819257662,-68.6089267801289,0,0,0
68,1,0,76.557229,41.304050,-73.909134,60.330934,9.663836,-70.570194,85.589476,66.735696,-74.449048,34.6608367765085,80.5063025132648,-57.0533715349259,56.4534258235572,71.0583220059077,-63.3865705072043,0,0,0
74,1,0,61.734499,9.673791,-68.924914,86.654889,66.667146,-73.507645,34.813024,80.472361,-57.081481,77.2894482549097,41.322553689217,-73.2016578462068,57.0744953046526,71.3728913333229,-62.8998792463111,0,0,0
125,1,0,35.249295,82.084425,-53.847704,60.520406,11.355786,-69.756839,77.052849,43.020596,-72.135130,57.1358755269656,72.3447055909126,-61.4923556359822,86.2038039594809,68.1617949711024,-72.1209659420695,0,0,0
128,1,0,35.477525,82.236724,-53.393348,61.366870,11.714407,-68.918578,77.758430,43.604703,-71.192710,87.2479078489883,68.9394285147865,-70.5754364471949,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77861,5,14,79.721273,29.985498,-50.744670,49.332506,129.053547,-46.686848,0.299055,97.382955,0.814677,27.2042151087381,134.852646342534,-37.8516495735271,-21.1523494714606,105.640480140953,-52.8228800047427,-0.543254305817506,122.135035587311,-47.7791910787856
77975,5,14,80.817822,30.668220,-48.626197,-1.739961,98.078545,5.620355,88.900802,61.893957,-59.065238,-29.5991151458287,77.5350640617349,-17.8015440460816,27.8193429448572,107.407423178687,12.8980066740448,61.6141133877569,101.369539546477,-3.43862015390206
77993,5,14,54.787381,129.109776,-43.547323,-2.258009,99.474948,6.206566,76.299480,33.369694,-41.871622,87.1919675610118,60.9778295733364,-60.4636764719324,-29.9804984846637,77.5406124828043,-17.7762221753,61.7243328712214,101.335898756965,-6.62046229165904
78053,5,14,55.181776,129.375538,-43.347963,-21.392640,104.044134,-55.057499,62.679053,99.842859,-7.622842,78.7753905459895,31.5444889193738,-46.6234949239803,-1.39853261904958,124.013898328683,-43.9532220754514,27.3736079318112,136.118517356196,-33.062961337576


# Class recognition without User label

In [18]:
df2 = df.drop(['User'], axis=1)
df2 = df2.drop(['Class'], axis=1)
labels_Class = df['Class']
X_train, X_test, y_train, y_test,  = train_test_split(df2, labels_Class, test_size=0.20, random_state=42)

clf = SVC(kernel='rbf', gamma="auto", C = 7)
clf.fit(X_train, y_train)
print(clf.score(X_test, y_test))

0.27839506172839507


# SVC gread search class recognition without user label

In [24]:
param_grid = {'C':[1,10,100,1000],
              'gamma':[1,0.1,0.001,0.0001], 
              'kernel':['sigmoid','rbf']}

grid = GridSearchCV(SVC(),param_grid,refit = True, verbose=2)

grid.fit(X_train,y_train)


Fitting 5 folds for each of 32 candidates, totalling 160 fits
[CV] END .......................C=1, gamma=1, kernel=sigmoid; total time=   2.4s
[CV] END .......................C=1, gamma=1, kernel=sigmoid; total time=   1.7s
[CV] END .......................C=1, gamma=1, kernel=sigmoid; total time=   1.7s
[CV] END .......................C=1, gamma=1, kernel=sigmoid; total time=   1.7s
[CV] END .......................C=1, gamma=1, kernel=sigmoid; total time=   2.5s
[CV] END ...........................C=1, gamma=1, kernel=rbf; total time=   5.0s
[CV] END ...........................C=1, gamma=1, kernel=rbf; total time=   3.7s
[CV] END ...........................C=1, gamma=1, kernel=rbf; total time=   3.7s
[CV] END ...........................C=1, gamma=1, kernel=rbf; total time=   5.0s
[CV] END ...........................C=1, gamma=1, kernel=rbf; total time=   3.7s
[CV] END .....................C=1, gamma=0.1, kernel=sigmoid; total time=   1.7s
[CV] END .....................C=1, gamma=0.1, k

GridSearchCV(estimator=SVC(),
             param_grid={'C': [1, 10, 100, 1000],
                         'gamma': [1, 0.1, 0.001, 0.0001],
                         'kernel': ['sigmoid', 'rbf']},
             verbose=2)

In [28]:
grid.best_params_
predic = grid.predict(X_test)
print(grid.score(X_test, y_test))
print(grid.best_params_)

0.7716049382716049
{'C': 10, 'gamma': 0.0001, 'kernel': 'rbf'}


# Hand owner recognition without Class label


In [32]:
df2 = df.drop(['User'], axis=1)
df2 = df2.drop(['Class'], axis=1)
labels_Users = df['User']
X_train, X_test, y_train, y_test,  = train_test_split(df2, labels_Users, test_size=0.20, random_state=42)

clf = SVC(kernel='rbf', gamma="auto", C = 7)
clf.fit(X_train, y_train)
print(clf.score(X_test, y_test))

0.15483870967741936


# Gread SearchHand owner recognition without Class label

In [33]:
param_grid = {'C':[1,10,100,1000],
              'gamma':[1,0.1,0.001,0.0001], 
              'kernel':['sigmoid','rbf']}

grid = GridSearchCV(SVC(),param_grid,refit = True, verbose=2)

grid.fit(X_train,y_train)

Fitting 5 folds for each of 32 candidates, totalling 160 fits
[CV] END .......................C=1, gamma=1, kernel=sigmoid; total time=   0.3s
[CV] END .......................C=1, gamma=1, kernel=sigmoid; total time=   0.3s
[CV] END .......................C=1, gamma=1, kernel=sigmoid; total time=   0.3s
[CV] END .......................C=1, gamma=1, kernel=sigmoid; total time=   0.3s
[CV] END .......................C=1, gamma=1, kernel=sigmoid; total time=   0.3s
[CV] END ...........................C=1, gamma=1, kernel=rbf; total time=   0.6s
[CV] END ...........................C=1, gamma=1, kernel=rbf; total time=   0.6s
[CV] END ...........................C=1, gamma=1, kernel=rbf; total time=   0.6s
[CV] END ...........................C=1, gamma=1, kernel=rbf; total time=   0.6s
[CV] END ...........................C=1, gamma=1, kernel=rbf; total time=   0.6s
[CV] END .....................C=1, gamma=0.1, kernel=sigmoid; total time=   0.3s
[CV] END .....................C=1, gamma=0.1, k

GridSearchCV(estimator=SVC(),
             param_grid={'C': [1, 10, 100, 1000],
                         'gamma': [1, 0.1, 0.001, 0.0001],
                         'kernel': ['sigmoid', 'rbf']},
             verbose=2)

In [34]:
grid.best_params_
predic = grid.predict(X_test)
print(grid.score(X_test, y_test))
print(grid.best_params_)

0.65
{'C': 10, 'gamma': 0.0001, 'kernel': 'rbf'}


# Random Forest

In [35]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification

In [36]:
dataset_path = 'allUsers.lcl.csv'

df = pd.read_csv(dataset_path)
#knn Cannot work with mining data, so replace them with a 0
df = df.replace('?', 0)
#df = df.dropna(thresh=len(df) * 0.10, axis=1)


# Class recognition with User label

In [ ]:
df2 = df.drop(['User'], axis=1)
labels_Users = df['User']
X_train, X_test, y_train, y_test,  = train_test_split(df2, labels_Users, test_size=0.20, random_state=42)

In [41]:
clf = RandomForestClassifier(max_depth=200, random_state=10)
clf.fit(X_train, y_train)
clf.score(X_test, y_test)

0.8032258064516129

# Gread Search